In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

import cv2
import matplotlib.pyplot as plt

In [ ]:
def convert_image_to_array(image_dir):
    try:
        image = cv2.imread(image_dir)
        if image is not None :
            return list(image)
        else :
            return np.array([])
    except Exception as e:
        print(f"Error : {e}")
        return None
    
def show_image(img):
    plt.figure(figsize=(50,50))
    plt.subplot(2,5,3)
    plt.grid(False)
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    #plt.xlabel(label_list[i])
#     plt.show()


In [ ]:
im = cv2.imread('../input/dandelionimages/Images/dandelion/IMG_1120.jpg')
im2 = cv2.resize(im, (128, 128))
# remove green
# for row in range(128):
#     for col in range(128):
#         cur_b = im2[row][col][0]
#         cur_g = im2[row][col][1]
#         cur_r = im2[row][col][2]
#         if(cur_g > 150 and cur_r > 150):
#             im2[row][col][1] = im2[row][col][2] = 255

#         else:
#             im2[row][col][0] = im2[row][col][1] = im2[row][col][2] = 0 
show_image(im)
show_image(im2)


In [ ]:
def compute_histogram_simple(img):
    im = img # ,0 แปลว่า อ่านแบบ Grayscale
    im = cv2.resize(im,(128,128)) # convert image to same size of 128x128
    #im2 = np.float32(im)/255.0 # map 0 - 255 to 0 - 1
    # [0, 32, 64, 96, 128, 160, 192, 224] * 3
    bins = [0]*8*3
    for row in range(128):
      for col in range(128):
        #color code
        current_pixel = im[row, col]
        #blue 0 -7
        #green 8 - 15
        #red 16 - 24
        for chanel in range(3):
            bin_index =  (8 * chanel) + current_pixel[chanel]//32
            bins[bin_index] += 1
        
    return bins

In [ ]:
def compute_histogram_zoning(img, dimension = 2):
    im = img
    im = cv2.resize(im, (128, 128))
    # [0, 32, 64, 96, 128, 160, 192, 224] * 3 and Part to {dimension} zone
    # 1 zone has 24 members
    col_per_zone = 128//dimension 
    bins = [0] * 8 * 3 * dimension * dimension
    # 0 1 2
    # 3 4 5
    # 6 7 8
    for row in range(128):
        for col in range(128):
            current_pixel = im[row, col]
            current_zone_x = row//col_per_zone
            current_zone_y = col//col_per_zone
            current_zone = dimension*current_zone_x + current_zone_y
            for chanel in range(3):
                current_bin_index_color = current_pixel[chanel]//32
                current_bin_index_chanel = 8 * chanel
                current_bin_index_zone = 24 * current_zone
                bin_index = current_bin_index_zone + current_bin_index_chanel + current_bin_index_color
#                 if(bin_index > 71):
#                     print(current_bin_index_zone, current_bin_index_chanel, current_bin_index_color, bin_index)
                bins[bin_index] += 1
    return bins

In [ ]:
def compute_distributed_weight(color):
    middle = [15.5 + 32 * i for i in range(8)]
    curr_middle = 0
    for i in range(7):
        if(color < middle[0]):
            return 0, 1, 0, 0
        elif(color > middle[-1]):
            return 0, 0, 1, 7
        elif(middle[i] > color):
            curr_middle = i
            break
    curr_lower = curr_middle
    curr_upper = curr_middle + 1
    weight_lower = abs(color - middle[curr_middle])/32
    weight_upper = abs(color - middle[curr_middle-1])/32
    return curr_lower, weight_lower, curr_upper, weight_upper

def compute_histogram_zonign_distributed(img, dimension = 2):
    im = img
    im = cv2.resize(im, (128, 128))
    # [0, 32, 64, 96, 128, 160, 192, 224] * 3 and Part to {dimension} zone
    # 1 zone has 24 members
    col_per_zone = 128//dimension 
    bins = [0] * 8 * 3 * dimension * dimension
    # 0 1 2
    # 3 4 5
    # 6 7 8
    for row in range(128):
        for col in range(128):
            current_pixel = im[row, col]
            current_zone_x = row//col_per_zone
            current_zone_y = col//col_per_zone
            current_zone = dimension*current_zone_x + current_zone_y
            for chanel in range(3):
                current_bin_index_chanel = 8 * chanel
                current_bin_index_zone = 24 * current_zone
                curr_lower, weight_lower, curr_upper, weight_upper = compute_distributed_weight(current_pixel[chanel])
                bin_index_lower = current_bin_index_zone + current_bin_index_chanel + curr_lower
                bin_index_upper = current_bin_index_zone + current_bin_index_chanel + curr_upper
                bins[bin_index_lower] += weight_lower
                bins[bin_index_upper] += weight_upper
                
                
    return bins 

In [ ]:
#Read File
dataset_path_dandilion = '../input/dandelionimages/Images/dandelion'
dataset_path_other = '../input/dandelionimages/Images/other'

dataset_dandilion_simple = []
dataset_other_simple = []

dataset_dandilion_zone = []
dataset_other_zone = []

dataset_dandilion_zone_d = []
dataset_other_zone_d = []


filename_dandilion = os.listdir(dataset_path_dandilion)
filename_other = os.listdir(dataset_path_other)


print('READING Dandelion...')
data_numbers = 620
term = (data_numbers * 0.05)//1

for i in range(data_numbers):
    if i == 0 : print('[', end="")
    elif i == data_numbers-1: print(']',end="")
    elif i % term == 0: print('o', end="")
#     print(f"[{i}] read file {filename_dandilion[i]}")
    im = cv2.imread(f'{dataset_path_other}/{filename_other[i]}')
    bins = compute_histogram_zonign_distributed(im)
    dataset_dandilion_simple.append(compute_histogram_simple(im))
    dataset_dandilion_zone.append(compute_histogram_zoning(im))
    dataset_dandilion_zone_d.append(compute_histogram_zonign_distributed(im))
print('COMPLETE!!')
print('READING Other...')
for i in range(data_numbers):
    if i == 0: print('[', end="")
    elif i == data_numbers-1: print(']',end="")
    elif i % term == 0: print('o', end="")
#     print(f"[{i}] read file {filename_other[i]}")
    im = cv2.imread(f'{dataset_path_other}/{filename_other[i]}')
    dataset_other_simple.append(compute_histogram_simple(im))
    dataset_other_zone.append(compute_histogram_zoning(im))
    dataset_other_zone_d.append(compute_histogram_zonign_distributed(im))
    
print('COMPLETE!!')

In [ ]:
from sklearn.model_selection import train_test_split
def merge_dan_with_other(first_data, second_data, test_size = 0.2):
    first_label = [1 for i in range(len(first_data))]
    second_label = [0 for i in range(len(second_data))]
    x_train_dan, x_test_dan, y_train_dan, y_test_dan = train_test_split(first_data,first_label,test_size=test_size)
    x_train_other, x_test_other, y_train_other, y_test_other = train_test_split(second_data,second_label,test_size=test_size)

    x_train = x_train_dan + x_train_other
    y_train = y_train_dan + y_train_other

    x_test = x_test_dan + x_test_other
    y_test = y_test_dan + y_test_other
    return x_train, x_test, y_train, y_test

x_train_s, x_test_s, y_train_s, y_test_s = merge_dan_with_other(dataset_dandilion_simple, dataset_other_simple)
x_train_z, x_test_z, y_train_z, y_test_z = merge_dan_with_other(dataset_dandilion_zone, dataset_other_zone)
x_train_zd, x_test_zd, y_train_zd, y_test_zd = merge_dan_with_other(dataset_dandilion_zone_d, dataset_other_zone_d)

In [ ]:
from sklearn import svm

model_s = svm.SVC()
model_z = svm.SVC()
model_zd = svm.SVC()
model_s.fit(x_train_s, y_train_s)
model_z.fit(x_train_z, y_train_z)
model_zd.fit(x_train_zd, y_train_zd)

In [ ]:
# Test Classifier
from sklearn.metrics import accuracy_score

y_pred_s = model_s.predict(x_test_s)
y_pred_z = model_z.predict(x_test_z)
y_pred_zd = model_zd.predict(x_test_zd)

acc_s = accuracy_score(y_test_s, y_pred_s)
acc_z = accuracy_score(y_test_z, y_pred_z)
acc_zd = accuracy_score(y_test_zd ,y_pred_zd)

print(f"simple acc:{acc_s}")
print(f"zone acc:{acc_z}")
print(f"zone distribute acc:{acc_zd}")